### Libraries

In [160]:
import numpy as np
import pandas as pd

#preprocess libraries used
import re
import nltk
from gensim.models import Word2Vec
from sklearn.preprocessing import MinMaxScaler
#ML libraries used



### Load Data

In [161]:
df = pd.read_csv('Anime_data.csv')
df = df[:10000]

### Show Data

In [162]:
display(df)
df.shape

,Anime_id,Title,Genre,Synopsis,Type,Producer,Studio,Rating,ScoredBy,Popularity,Members,Episodes,Source,Aired,Link
0,1,Cowboy Bebop,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...","In the year 2071, humanity has colonized sever...",TV,['Bandai Visual'],['Sunrise'],8.81,363889.0,39.0,704490.0,26.0,Original,"Apr 3, 1998 to Apr 24, 1999",https://myanimelist.net/anime/1/Cowboy_Bebop
1,5,Cowboy Bebop: Tengoku no Tobira,"['Action', 'Space', 'Drama', 'Mystery', 'Sci-Fi']","Another day, another bounty—such is the life o...",Movie,"['Sunrise', 'Bandai Visual']",['Bones'],8.41,111187.0,475.0,179899.0,1.0,Original,"Sep 1, 2001",https://myanimelist.net/anime/5/Cowboy_Bebop__...
2,6,Trigun,"['Action', 'Sci-Fi', 'Adventure', 'Comedy', 'D...","Vash the Stampede is the man with a $$60,000,0...",TV,['Victor Entertainment'],['Madhouse'],8.31,197451.0,158.0,372709.0,26.0,Manga,"Apr 1, 1998 to Sep 30, 1998",https://myanimelist.net/anime/6/Trigun
3,7,Witch Hunter Robin,"['Action', 'Magic', 'Police', 'Supernatural', ...",Witches are individuals with special powers li...,TV,['Bandai Visual'],['Sunrise'],7.34,31875.0,1278.0,74889.0,26.0,Original,"Jul 2, 2002 to Dec 24, 2002",https://myanimelist.net/anime/7/Witch_Hunter_R...
4,8,Bouken Ou Beet,"['Adventure', 'Fantasy', 'Shounen', 'Supernatu...",It is the dark century and the people are suff...,TV,NaN,['Toei Animation'],7.04,4757.0,3968.0,11247.0,52.0,Manga,"Sep 30, 2004 to Sep 29, 2005",https://myanimelist.net/anime/8/Bouken_Ou_Beet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,31980,Okusama ga Seitokaichou!: Seitokaichou to Ofur...,"['Comedy', 'Romance', 'Ecchi', 'Shounen']",Bundled with the limited edition ninth manga v...,OVA,NaN,['Seven'],6.89,13242.0,2612.0,23673.0,1.0,Manga,"Jan 27, 2016",https://myanimelist.net/anime/31980/Okusama_ga...
9996,31984,Lupin Shanshei Pilot,"['Parody', 'Comedy']",NaN,Special,NaN,['TMS Entertainment'],5.69,76.0,10800.0,370.0,1.0,Unknown,"Dec 19, 2012",https://myanimelist.net/anime/31984/Lupin_Shan...
9997,31988,Hibike! Euphonium 2,"['Music', 'School', 'Drama']",Following their success in the qualifying roun...,TV,"['Lantis', 'Pony Canyon', 'Rakuonsha']",['Kyoto Animation'],8.29,47496.0,873.0,96929.0,13.0,Novel,"Oct 6, 2016 to Dec 29, 2016",https://myanimelist.net/anime/31988/Hibike_Eup...
9998,31989,Hibike! Euphonium Movie 1: Kitauji Koukou Suis...,"['Music', 'School']",After swearing off music due to an incident at...,Movie,"['Lantis', 'Shochiku']",['Kyoto Animation'],7.45,3358.0,3541.0,12549.0,1.0,Novel,"Apr 23, 2016",https://myanimelist.net/anime/31989/Hibike_Eup...


(10000, 15)

In [163]:
df.columns

Index(['Anime_id', 'Title', 'Genre', 'Synopsis', 'Type', 'Producer', 'Studio',
       'Rating', 'ScoredBy', 'Popularity', 'Members', 'Episodes', 'Source',
       'Aired', 'Link'],
      dtype='object')

### Preprocess Data

In [164]:
COLUMNS = [
    'Title',
    'Genre',
    'Synopsis',
    'Type',
    'Producer',
    'Studio',
    'Rating',
#    'ScoredBy',
#    'Popularity',
#    'Members',
#    'Episodes',
#    'Source',
#    'Aired'
]

df_preprocess = df[COLUMNS]
df_preprocess

,Title,Genre,Synopsis,Type,Producer,Studio,Rating
0,Cowboy Bebop,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...","In the year 2071, humanity has colonized sever...",TV,['Bandai Visual'],['Sunrise'],8.81
1,Cowboy Bebop: Tengoku no Tobira,"['Action', 'Space', 'Drama', 'Mystery', 'Sci-Fi']","Another day, another bounty—such is the life o...",Movie,"['Sunrise', 'Bandai Visual']",['Bones'],8.41
2,Trigun,"['Action', 'Sci-Fi', 'Adventure', 'Comedy', 'D...","Vash the Stampede is the man with a $$60,000,0...",TV,['Victor Entertainment'],['Madhouse'],8.31
3,Witch Hunter Robin,"['Action', 'Magic', 'Police', 'Supernatural', ...",Witches are individuals with special powers li...,TV,['Bandai Visual'],['Sunrise'],7.34
4,Bouken Ou Beet,"['Adventure', 'Fantasy', 'Shounen', 'Supernatu...",It is the dark century and the people are suff...,TV,NaN,['Toei Animation'],7.04
...,...,...,...,...,...,...,...
9995,Okusama ga Seitokaichou!: Seitokaichou to Ofur...,"['Comedy', 'Romance', 'Ecchi', 'Shounen']",Bundled with the limited edition ninth manga v...,OVA,NaN,['Seven'],6.89
9996,Lupin Shanshei Pilot,"['Parody', 'Comedy']",NaN,Special,NaN,['TMS Entertainment'],5.69
9997,Hibike! Euphonium 2,"['Music', 'School', 'Drama']",Following their success in the qualifying roun...,TV,"['Lantis', 'Pony Canyon', 'Rakuonsha']",['Kyoto Animation'],8.29
9998,Hibike! Euphonium Movie 1: Kitauji Koukou Suis...,"['Music', 'School']",After swearing off music due to an incident at...,Movie,"['Lantis', 'Shochiku']",['Kyoto Animation'],7.45


In [165]:
def preprocess_text(text):
    # Remove ' \r\n \r\n' plus last sentence
    text = re.sub(r'\s*(\r\n\s*)+|\[Written by .*?\]+|\(Source: .*?\)\s*', '', text)
    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Tokenize the text into words
    words = nltk.word_tokenize(text)
    # Remove stop words
    words = [word for word in words if word not in nltk.corpus.stopwords.words('english')]
    return words

def get_vector(text):
    tokens = [token for token in text]
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    return sum(vectors)/len(vectors)

def get_title_vector(title):
    words = title.lower().split()
    vectors = [model.wv[word] for word in words if word in model.wv.key_to_index]
    if len(vectors) > 0:
        return sum(vectors) / len(vectors)
    else:
        return np.zeros(model.vector_size)

##### Preprocess de la colonne Genre

In [166]:
df_genre = df[['Title','Genre']]

df_genre['Genre'] = df_genre['Genre'].apply(lambda x: "['NoGenre']" if pd.isna(x) else x)
df_genre['Genre'] = df_genre['Genre'].apply(lambda x: eval(x))

df_genre['Genre'] = df_genre['Genre'].apply(lambda x: '|'.join(x))

genres = df_genre['Genre'].str.get_dummies('|')

df_preprocess = pd.concat([df_preprocess, genres], axis=1).drop('Genre', axis=1)

C:\Users\Armand\AppData\Local\Temp/ipykernel_50156/2526877318.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genre['Genre'] = df_genre['Genre'].apply(lambda x: "['NoGenre']" if pd.isna(x) else x)
C:\Users\Armand\AppData\Local\Temp/ipykernel_50156/2526877318.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genre['Genre'] = df_genre['Genre'].apply(lambda x: eval(x))
C:\Users\Armand\AppData\Local\Temp/ipykernel_50156/2526877318.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy 

##### Preprocess de la colonne Producer

In [167]:
df_producer = df[['Title','Producer']]

df_producer['Producer'] = df_producer['Producer'].apply(lambda x: "['NoProducer']" if pd.isna(x) else x)
df_producer['Producer'] = df_producer['Producer'].apply(lambda x: eval(x))

df_producer['Producer'] = df_producer['Producer'].apply(lambda x: '|'.join(x))

producers = df_producer['Producer'].str.get_dummies('|')

df_preprocess = pd.concat([df_preprocess, producers], axis=1).drop('Producer', axis=1)

C:\Users\Armand\AppData\Local\Temp/ipykernel_50156/3710069928.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_producer['Producer'] = df_producer['Producer'].apply(lambda x: "['NoProducer']" if pd.isna(x) else x)
C:\Users\Armand\AppData\Local\Temp/ipykernel_50156/3710069928.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_producer['Producer'] = df_producer['Producer'].apply(lambda x: eval(x))
C:\Users\Armand\AppData\Local\Temp/ipykernel_50156/3710069928.py:6: SettingWithCopyWarning: 
A value is 

##### Preprocess de la colonne Studio

In [168]:
df_studio = df[['Title','Studio']]

df_studio['Studio'] = df_studio['Studio'].apply(lambda x: "['NoStudio']" if pd.isna(x) else x)
df_studio['Studio'] = df_studio['Studio'].apply(lambda x: eval(x))

df_studio['Studio'] = df_studio['Studio'].apply(lambda x: '|'.join(x))

studio = df_studio['Studio'].str.get_dummies('|')
studio
df_preprocess = pd.concat([df_preprocess, studio], axis=1).drop('Studio', axis=1)

C:\Users\Armand\AppData\Local\Temp/ipykernel_50156/2030060948.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_studio['Studio'] = df_studio['Studio'].apply(lambda x: "['NoStudio']" if pd.isna(x) else x)
C:\Users\Armand\AppData\Local\Temp/ipykernel_50156/2030060948.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_studio['Studio'] = df_studio['Studio'].apply(lambda x: eval(x))
C:\Users\Armand\AppData\Local\Temp/ipykernel_50156/2030060948.py:6: SettingWithCopyWarning: 
A value is trying to be set o

##### Preprocess de la colonne Type

In [169]:
df_preprocess = pd.get_dummies(df_preprocess, columns = ["Type"])

##### Preprocess de la colonne Synospsis

In [170]:
%%time
df_synospsis = df_preprocess[['Title', 'Synopsis']]

df_synospsis['Synopsis'] = df_synospsis['Synopsis'].fillna('NoSynopsis')
df_synospsis['Synopsis'] = df_synospsis['Synopsis'].apply(preprocess_text)

<timed exec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Wall time: 3min 10s


<timed exec>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [171]:
# df_synospsis

In [172]:
model = Word2Vec(df_synospsis['Synopsis'], vector_size=100, window=5, min_count=1, workers=4)

In [173]:
%%time
df_synospsis['vector_synopsis'] = df_synospsis['Synopsis'].apply(get_vector)
df_preprocess['Synopsis'] = df_synospsis['vector_synopsis']

Wall time: 683 ms


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


##### Preprocess de la colonne Title

In [174]:
%%time

model = Word2Vec(df_preprocess['Title'].apply(preprocess_text), vector_size=100, window=5, min_count=1, workers=4)

df_preprocess['Title'] = df_preprocess['Title'].apply(get_title_vector)

scaler = MinMaxScaler()

df_preprocess['Title'] = scaler.fit_transform(df_preprocess['Title'].tolist())

Wall time: 12.5 s


In [175]:
df_preprocess

,Title,Synopsis,Rating,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,...,ixtl,teamKG,ufotable,Type_Movie,Type_Music,Type_ONA,Type_OVA,Type_Special,Type_TV,Type_Unknown
0,0.552966,"[-0.18001947, 0.5122388, 0.24023183, 0.1490239...",8.81,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1,0.421723,"[-0.13932915, 0.4747972, 0.22125441, 0.1633791...",8.41,1,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
2,0.742857,"[-0.14287421, 0.42321265, 0.21097961, 0.137622...",8.31,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
3,0.518601,"[-0.20528497, 0.5842097, 0.29815558, 0.1713274...",7.34,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
4,0.436056,"[-0.16506654, 0.6109541, 0.3345055, 0.17323433...",7.04,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.432945,"[-0.4184853, 0.8125163, 0.42066154, 0.02799009...",6.89,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
9996,0.646539,"[-0.0066730664, -0.0015420723, 0.006479461, -0...",5.69,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9997,0.419591,"[-0.19097899, 0.5432713, 0.15612336, 0.2265137...",8.29,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
9998,0.448641,"[-0.20853887, 0.59361243, 0.11582668, 0.252557...",7.45,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


### Train model